In [1]:
import sys
sys.path.insert(0,'data')
import requests

import utils_db
import utils
import dialogs
import utils_toloka
import time
import vk_api
import config
import numpy as np
import pandas as pd

import logging
logger = logging.getLogger('toloka_loader')
handler = logging.FileHandler('logs/toloka_loader.log',encoding = "UTF-8")
formatter = logging.Formatter('------------\n %(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter); logger.addHandler(handler)

dic = {'hot':'счетчик горячей воды','cold':'счетчик холодной воды',
       'electricity':'счетчик электроэнергии','gas':'счетчик газа','other':'определить не удалось'}

def add_new_img_to_toloka(new_images):
    result_post_request = requests.post(url = utils_toloka.url_create_tasks,headers = utils_toloka.headers,
                                        json = utils_toloka.create_json_for_tasks(new_images))
    for task in result_post_request.json()['items'].values():
        image_id = int(task['input_values']['image'][:-4].split('/')[-1])
        utils_db.add_toloka_task_id_in_db(image_id,task['id'])

In [2]:
def check_reviewing_images(reviewing_images):
    for reviewing_img in reviewing_images:
        try:
            result_get_request = requests.get(url=utils_toloka.url_task_execution_check(
                    reviewing_img['id_toloka_task']),headers = utils_toloka.headers).json()
            if len(result_get_request['items']) >= 3:
                type_list = list(map(lambda t : t['solutions']
                                     [np.where(np.array(list(map(lambda x: x['id'],t['tasks']))) 
                                    == reviewing_img['id_toloka_task'])[0][0]]
                                     ['output_values']['type'],result_get_request['items']))
                type_res = pd.Series(type_list).value_counts().index[0]
                result_list = list(map(lambda t : t['solutions'][
                                    np.where(np.array(list(map(lambda x: x['id'],t['tasks']))) 
                                   == reviewing_img['id_toloka_task'])[0][0]]
                                       ['output_values']['result'],result_get_request['items']))
                result_res = pd.Series(result_list).value_counts().index[0]
                meters_list = list(map(lambda t : t['solutions']
                                       [np.where(np.array(list(map(lambda x: x['id'],t['tasks']))) 
                                    == reviewing_img['id_toloka_task'])[0][0]]
                                       ['output_values']['meters_data'],result_get_request['items']))
                meters_res = pd.Series(meters_list).value_counts().index[0]
                print(type_res,result_res,meters_res)
                utils_db.add_review_in_db(reviewing_img['image_id'],str([type_res,result_res,meters_res]))
                if result_res == 'no':
                    mesg = 'На изображении счетчик не обнаружен.'
                elif result_res == 'some':
                    mesg = 'На изображении несколько счетчиков. Пришли изображение только с одним счетчиком.'
                elif result_res == 'bad':
                    mesg = 'Изображение плохого качества. Не удается распознать показания счетчиков.'
                elif result_res == 'good':
                    mesg = 'Тип: %s.\nПоказания: %g' %(dic[type_res],meters_res)
                print(mesg)
                utils.send_msg(reviewing_img['user_id'],mesg)
        except Exception as e:
            if type(e) == vk_api.exceptions.ApiError and e.code == 901:
                #user blocked us
                pass
            else:
                logger.exception("error toloka_loader_hh ")
                utils.send_me_error("error toloka_loader_hh ", e)

In [ ]:
while True:
    try:
        st = time.time()
        new_images = utils_db.select_new_img_to_toloka()
        if len(new_images) > 0:
            add_new_img_to_toloka(new_images)
        reviewing_images = utils_db.select_reviewing_images()
        if len(reviewing_images) > 0:
            check_reviewing_images(reviewing_images)
        time.sleep(max(2 - (time.time()-st),0))
    except KeyboardInterrupt:
        break
    except Exception as e:
        logger.exception("error toloka_loader ")
        utils.send_me_error("error toloka_loader ", e)

electricity good 0.29
Тип: счетчик электроэнергии.
Показания: 0.29
